<a href="https://colab.research.google.com/github/lourencocavalcante/CAP-Dissertation-Complementary-studies/blob/main/Down_Data_MP3000A_ATTO_Campina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[<img src="https://github.com/lourencocavalcante/LogosINPE/blob/main/logoinpe.png?raw=true" width = 500 align="left">](https://www.gov.br/inpe/pt-br)

[<img src="https://github.com/lourencocavalcante/LogosINPE/blob/main/LogoCAP.png?raw=true" width = 300 align="right">](http://www.inpe.br/posgraduacao/cap/)

# <span style="color:#336699; text-align: center;">MP3000A: Download e visualização dos dados  (Index of /ftp/public/ATTO/Campina/MP3000)</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 110%;">
    <b>Download e visualização dos dados</b>
    <br/>
    <b>Docente:</b> Dr. Alan James
    <br/><br/>
    <b>Discente:</b> Lourenço José Cavalcante Neto
    <br/>
    <b>E-mail:</b> <a href="mailto:lourenco.cavalcante@ifto.edu.b">lourenco.cavalcante@ifto.edu.br</a>

</div>

<br/>

<div style="text-align: justify;  margin-left: 20%; margin-right: 20%;">
<b>Objetivo: </b> O principal objetivo deste caderno notebook é baixar (via FTP) e visualizar os dados do Radiômetro de microondas terreste (Modelo MP-3000A), instalado no sítio Campina (S 02 10.9142 W 059 1.3063), no qual faz parte da composição do projeto Amazonian Tall Tower Observatory  (ATTO).
</div>

<span style="text-align: center;"> Este caderno notebook também pode ser visualizado no **Github**,  [Clicando aqui](https://github.com/lourencocavalcante/CAP-Dissertation-Complementary-studies.git) </span>

# **Radiômetro de microondas terreste - Modelo MP3000A**

O radiômetro MP3000A mede a temperatura de brilho atmosférico, a partir da qual podem ser derivados perfis verticais de temperatura e umidade de alta resolução e perfis de água líquida de baixa resolução da superfície até 10 km de altura. Esta sondagem atmosférica é útil para previsão do tempo e previsão de clima (Radiometrics, 2008; Madhulatha et al, 2013). As recuperações de perfil derivadas das medições do radiômetro são avaliadas por uma rede neural artificial cuja precisão diminui à medida que a altitude aumenta, diferentemente de radiossondas.

O instrumento incorpora dois subsistemas de microondas que compartilham a mesma antena e sistema apontador. O subsistema de perfil de temperatura usa observações de temperaturas de brilho do céu em frequências selecionadas que variam de 51 a 59 GHz (complexo de absorção de oxigênio). O subsistema de perfis de vapor de água usa observações de temperaturas de brilho do céu em frequências entre 22 e 30 GHz (linha de absorção de vapor de água) (Solheim et al, 1998; Ware et al, 2003).

O MP3000A conta com sensores meteorológicos de superfície, que medem temperatura do ar, umidade relativa, pressão barométrica e ocorrência de chuva a partir de um sensor de chuva. Além disso, um termômetro infravermelho apontando para o zênite é montado internamente para melhorar a precisão dos perfis de densidade de vapor d'água e água líquida.

Para medir o ganho do sistema, o instrumento possui sinal gerado por fontes de ruído (diodos), durante seu funcionamento normal. Quando os diodos são ativados, um nível de temperatura de brilho calibrado é adicionado à cena em cada observação. Além disso, o MP3000A utiliza um alvo de referência interno, ajustado à temperatura ambiente (usado como um corpo negro), que fornece padrões primários conhecidos para cada medição e trabalha em conjunto com um alvo criogênico externo que fornece padrões absolutos de calibração para os diodos.



---



# **Importa as bibliotecas**

In [21]:
!pip install netCDF4 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from datetime import datetime, timedelta

import matplotlib
matplotlib.rcParams.update({'font.size':22})
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap # interpolação linear para cores
from netCDF4 import Dataset  
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")

# **Conexão com o Drive e definição do caminho da pasta salvar os dados**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define o caminho do drive. Pasta para a saída das Figuras geradas que serão usadas na CNN
path_data_mp3000a = f'/content/drive/MyDrive/MestradoCAP/2023/Dados-Campina-ATTO/MP-3000A'



---



## **Baixando os dados:**



Vamos definir a data e a hora dos dados para download e posterior visualização.

In [53]:
# O ano, mês, dia, hora, minuto e segundo são definidos como variáveis, 
# enquanto o nível é definido como um intervalo de números. 

ano = '2021'
mes = '07'
dia = '27'
hora = '19'
min = '43'
seg = '20'
#files_list = ['healthstatus', 'lv0', 'lv1', 'lv2', 'tip']
files_list = ['lv0', 'lv1', 'lv2']

**level0:**
Neste arquivo estão os dados do ambiente próximo à superfície (Dados brutos do sensor).

**level1:**
Neste arquivos estão os dados das medidas dos canais do microonda (Temperaturas de brilho).

**level2:**
Neste arquivo estão as variáveis estimadas associadas a parâmetros de  (Recuperação de perfil).

**tip:**
Arquivos de dados de calibração

Ou seja...

**Arquivo de Nível 0:** Os arquivos de nível 0 contêm dados brutos, não processados, em unidades de engenharia. Um arquivo de nível 0 é produzido para todos os modos de operação e todas as opções que podem ser selecionadas do menu principal. Os arquivos de nível 0 contêm 100% das informações necessárias para reprocessar os dados brutos com informações de calibração ou algoritmos alternativos. Os arquivos de nível 0 contêm os seguintes tipos de registros:

**Arquivo de Nível 1:** Os arquivos de nível 1 contêm temperaturas de brilho em tempo real para cada canal especificado no arquivo de configuração. Os arquivos de nível 1 em tempo real são produzidos a partir de dados de nível 0 contemporâneos e informações de calibração no arquivo de configuração. Os arquivos de nível 1 contêm os seguintes tipos de registros:

**Arquivo de Nível 2:** Os arquivos de nível 2 contêm registros de recuperações em tempo real de temperatura (K), vapor d'água (g/m3), umidade relativa (%) e água líquida (g/m3) perfis. As recuperações são produzidas usando os dados de nível 1 contemporâneos e os arquivos de rede neural especificados no arquivo de configuração.

Vamos baixar os dados.

In [54]:
# Endereço do FTP do ATTO/Campina/MP3000
ftp_ATTO_campina = 'ftp.lfa.if.usp.br'
# ---------------------------------------------------------- #
#              BAIXANDO DADOS DO MP-3000A
# ---------------------------------------------------------- #

for file_name in files_list:

  # Nome do arquivo
  file_ir =  f'{ftp_ATTO_campina}/ftp/public/ATTO/Campina/MP3000/{ano}-{mes}-{dia}_{hora}-{min}-{seg}_{file_name}.csv'

  # Download dos arquivos
  !wget -c {file_ir} -P /content/drive/MyDrive/MestradoCAP/2023/Dados-Campina-ATTO/MP-3000A

--2023-01-30 19:48:09--  http://ftp.lfa.if.usp.br/ftp/public/ATTO/Campina/MP3000/2021-07-27_19-43-20_lv0.csv
Resolving ftp.lfa.if.usp.br (ftp.lfa.if.usp.br)... 143.107.131.83
Connecting to ftp.lfa.if.usp.br (ftp.lfa.if.usp.br)|143.107.131.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 897178 (876K) [text/csv]
Saving to: ‘/content/drive/MyDrive/MestradoCAP/2023/Dados-Campina-ATTO/MP-3000A/2021-07-27_19-43-20_lv0.csv’

2021-07-27_19-43-20 100%[===================>] 876.15K  1.23MB/s    in 0.7s    

2023-01-30 19:48:10 (1.23 MB/s) - ‘/content/drive/MyDrive/MestradoCAP/2023/Dados-Campina-ATTO/MP-3000A/2021-07-27_19-43-20_lv0.csv’ saved [897178/897178]

--2023-01-30 19:48:10--  http://ftp.lfa.if.usp.br/ftp/public/ATTO/Campina/MP3000/2021-07-27_19-43-20_lv1.csv
Resolving ftp.lfa.if.usp.br (ftp.lfa.if.usp.br)... 143.107.131.83
Connecting to ftp.lfa.if.usp.br (ftp.lfa.if.usp.br)|143.107.131.83|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length:

In [55]:
os.chdir(f'{path_data_mp3000a}')
full_direc_mp3000a = os.listdir()

In [56]:
full_direc_mp3000a

['2021-07-27_19-43-20_lv0.csv',
 '2021-07-27_19-43-20_lv1.csv',
 '2021-07-27_19-43-20_lv2.csv']

In [ ]:
for item_mp3000a in full_direc_mp3000a:
  #Carregar o Dataset e criar o Dataframe
  data_mp3000a = pd.read_csv(f'/content/drive/MyDrive/MestradoCAP/2023/Dados-Campina-ATTO/MP-3000A/{item_mp3000a}', delimiter=',',error_bad_lines = False)

  print(item_mp3000a)
  print(data_mp3000a)

# **Referências**

1. [...]